# Roteiro de Laboratório

## Sensor Escolhido: Câmera (webcam do notebook)

---

### Como realizar o roteiro

Clique no botão de play no lado esquerdo da célula de código para rodar aquela célula especificamente.

<img src="image-1.png" width="400">

A imagem da câmera aparecerá na tela.

<img src="image.png" width="400">
 

> ### Após terminar o exemplo, pressione Esc ou "Q" para fechar a janela e continuar no roteiro.

---

### Exemplo 1: Imagem sem filtros

Para acessar a câmera do notebook, utiliza-se a biblioteca OpenCV.

Esta é a imagem da câmera sem aplicação de filtros.

In [1]:
import cv2
from utils import flip_and_show_image

# Inicia a captura da webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar frame")
        break

    key = flip_and_show_image("Imagem sem filtros", frame)
    if key in (27, ord("q")):
        break

cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/lucasfv/Workspaces/roteiro-lab-robotica/venv/lib/python3.12/site-packages/cv2/qt/plugins"


### Exemplo 2: Imagem com Suavização (Blur Gaussiano)

Este filtro reduz ruído de alta frequência na imagem, tentando preservas os contornos da imagem.

In [3]:
import cv2
import numpy as np
from utils import flip_and_show_image

# Inicia a captura da webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar frame")
        break

    g_blurred_image = cv2.GaussianBlur(frame, (5, 5), 0)
    
    image = np.hstack((g_blurred_image, frame))
    key = flip_and_show_image("Imagem Original | Imagem com Blur Gaussiano", image)
    if key in (27, ord("q")):
        break

cap.release()
cv2.destroyAllWindows()

### Exemplo 3: Binarização da Imagem por Cores

Binarização por cores é uma técnica que transforma uma imagem colorida em uma imagem preta e branca, destacando apenas os pixels que estão dentro de uma faixa específica de cor, como se fosse um filtro que "enxerga" só uma cor e apaga o resto.

Neste exemplo, é possível criar um filtro de cor através de *sliders*. Os valores finais são mostrados após fechar a janela.

In [5]:
import cv2
import numpy as np

from utils import flip_and_show_image

def nothing(x):
    pass

# Cria janela com trackbars
cv2.namedWindow("Trackbars")
cv2.resizeWindow("Trackbars", 400, 300)

cv2.createTrackbar("H Min", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("H Max", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("S Min", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("S Max", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("V Min", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("V Max", "Trackbars", 255, 255, nothing)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # espelha para facilitar calibração
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Lê os valores dos trackbars
    h_min = cv2.getTrackbarPos("H Min", "Trackbars")
    h_max = cv2.getTrackbarPos("H Max", "Trackbars")
    s_min = cv2.getTrackbarPos("S Min", "Trackbars")
    s_max = cv2.getTrackbarPos("S Max", "Trackbars")
    v_min = cv2.getTrackbarPos("V Min", "Trackbars")
    v_max = cv2.getTrackbarPos("V Max", "Trackbars")

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])

    # Cria a máscara
    binary_mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(frame, frame, mask=binary_mask)

    # Concatena as imagens lado a lado
    mask_bgr = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
    stacked = np.hstack((result, mask_bgr))

    cv2.imshow("Mascara | Resultado", stacked)
    key = cv2.waitKey(1)
    if key in (27, ord("q")):  # tecla ESC para sair
        break

print("Valores finais:")
print("H Min:", h_min)
print("H Max:", h_max)
print("S Min:", s_min)
print("S Max:", s_max)
print("V Min:", v_min)
print("V Max:", v_max)

cap.release()
cv2.destroyAllWindows()

Valores finais:
H Min: 0
H Max: 179
S Min: 0
S Max: 255
V Min: 0
V Max: 255


### Exemplo 4: Operações Morfológicas

Depois da binarização por cor, a imagem binária ainda pode conter:
- Buracos internos no objeto.
- Pequenos pontos isolados (ruído).

**Erosão** remove ruído pequeno.

**Dilatação** reconstrói o objeto após a erosão.

---

Neste exemplo, a imagem é *binarizada* através da intensidade de luz e aplicadas operações morfológicas.

In [ ]:
import cv2
from utils import flip_and_show_image

# Inicia a captura da webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar frame")
        break

    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)[1]

    kernel = np.ones((7, 7), np.uint8)
    binary_mask = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

    binary_mask = cv2.bitwise_and(frame, frame, mask=binary_mask)
    binary = cv2.bitwise_and(frame, frame, mask=binary)

    # binary = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    image = np.hstack((binary_mask, binary))
    key = flip_and_show_image(
        "Imagem Binarizada | Imagem Binarizada com Operacoes Morfologicas", image
    )
    if key in (27, ord("q")):
        break

cap.release()
cv2.destroyAllWindows()


### Exemplo 5: Filtro de Área de Pixels

Também é possível selecionar apenas os objetos/contornos com uma área mínima e máxima de pixels.

In [7]:
import cv2
from utils import flip_and_show_image

# Inicia a captura da webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar frame")
        break

    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)[1]

    # mask = cv2.bitwise_and(frame, frame, mask=binary)
    # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 1000 and area < 100000:
            filtered_contours.append(contour)

    # mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    filtered_image = np.zeros_like(frame)
    filtered_image = cv2.drawContours(
        filtered_image, filtered_contours, -1, (255, 255, 255), -1
    )

    filtered_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
    filtered_image = cv2.bitwise_and(frame, frame, mask=filtered_image)

    binary_mask = cv2.bitwise_and(frame, frame, mask=binary)
    image = np.hstack((filtered_image, binary_mask))
    key = flip_and_show_image(
        "Imagem Binarizada x Imagem Binarizada com Operacoes Morfologicas", image
    )
    if key in (27, ord("q")):
        break

cap.release()
cv2.destroyAllWindows()


### Exercício 1

Utilizando o Exemplo 3, encontre as faixas de valores HSV que destacam a bola de tênis na imagem.

### Exercício 2

- Simula um sistema de Visual Servoing.

- A posição do objeto serve como sensor visual para controle.

- O jogador observa em tempo real o erro de posição e o ruído do processo.

- Ideal para explorar estabilidade, latência e robustez em controle por visão.

#### Instruções do Jogo
1. Use um objeto colorido (ex: bola de tênis).

1. Mantenha o objeto dentro do círculo por 3 segundos.

1. Ao conseguir:

    - Ganha 1 ponto.

    - O círculo muda de posição e fica menor.

1. O jogo termina quando o raio atinge o mínimo.


---

#### O Sistema é Bom o Suficiente para Realizar Controle?

---

#### Scoreboard

**Grupo 1**: 16 pontos

**...**: ?

**Grupo A**: ?

**Mecazord**: ?

**Robô 5R**: ?

**Um Grupo**: ?


In [3]:
import cv2
import numpy as np
import time
import random
from utils import flip_and_show_image

# Limites da cor (ajustáveis conforme o objeto)
lower_color = np.array([18, 70, 64])
upper_color = np.array([35, 255, 255])

# Webcam
cap = cv2.VideoCapture(0)

# Inicialização do círculo de tolerância
radius = 80
min_radius = 2
circle_pos = (320, 240)  # inicial (centro)
hold_time_required = 3  # segundos
hold_start_time = None
score = 0

font = cv2.FONT_HERSHEY_SIMPLEX

def new_circle_position(frame_shape):
    h, w = frame_shape[:2]
    x = random.randint(radius, w - radius)
    y = random.randint(radius, h - radius)
    return (x, y)

def circularity(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return 0  # evita divisão por zero
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    return circularity

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip para efeito espelho (opcional)
    frame = cv2.flip(frame, 1)

    # Conversão para HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Filtragem por Blur
    mask = cv2.GaussianBlur(hsv, (5, 5), 0)

    # Filtragem por Cor
    mask = cv2.inRange(mask, lower_color, upper_color)

    # Filtragem por Operação Morfológica
    kernel = np.ones((7, 7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((7, 7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Filtragem por Área de Pixels
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 40000 and area > 1000 and circularity(contour) > 0.5:
            filtered_contours.append(contour)

    # print(filtered_contours)
    object_center = None
    if filtered_contours:
        best_contour = sorted(filtered_contours, key=circularity, reverse=True)[0]

        blank = np.zeros_like(mask)
        mask = cv2.drawContours(blank, [best_contour], -1, (255, 255, 255), -1)

        # Momentos para detectar centro do objeto
        M = cv2.moments(mask)
        if M["m00"] > 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            object_center = (cx, cy)

            # Desenha centro do objeto
            cv2.circle(frame, object_center, 5, (255, 0, 0), -1)

            # Desenha linha até centro do círculo
            cv2.line(frame, circle_pos, object_center, (0, 255, 255), 2)

    # Desenha círculo de tolerância
    if object_center:
        dist = np.linalg.norm(np.array(object_center) - np.array(circle_pos))
        inside = dist <= radius
    else:
        inside = False

    circle_color = (0, 255, 0) if inside else (0, 0, 255)
    cv2.circle(frame, circle_pos, radius, circle_color, 2)

    # Lógica de tempo e pontuação
    now = time.time()
    if inside:
        if hold_start_time is None:
            hold_start_time = now
        elif now - hold_start_time >= hold_time_required:
            score += 1
            print(f"Ponto! Pontuação atual: {score}")
            hold_start_time = None
            if radius > 20:
                radius = max(radius - 10, min_radius)
            elif radius > 10:
                radius = max(radius - 3, min_radius)
            elif radius > 1:
                radius = max(radius - 1, min_radius)
            circle_pos = new_circle_position(frame.shape)
    else:
        hold_start_time = None

    # Feedback na tela
    if hold_start_time:
        elapsed = now - hold_start_time
        cv2.putText(frame, f"Segure por: {hold_time_required - elapsed:.1f}s", (10, 30), font, 0.8, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "Mantenha o objeto no circulo!", (10, 30), font, 0.8, (0, 0, 255), 2)

    cv2.putText(frame, f"Pontos: {score}", (10, 60), font, 0.8, (255, 255, 255), 2)

    # Fim de jogo
    if radius == min_radius:
        cv2.putText(frame, "JOGO ENCERRADO!", (200, 240), font, 1, (0, 255, 0), 3)

    # Mostrar resultado
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    frame = cv2.flip(frame, 1)
    mask = cv2.flip(mask, 1)
    result = np.hstack((mask, frame))
    
    key = flip_and_show_image("Jogo Visual Servoing | Mascara de Filtros", result)
    if key in (27, ord("q")):
        break

cap.release()
cv2.destroyAllWindows()


Ponto! Pontuação atual: 1
Ponto! Pontuação atual: 2
Ponto! Pontuação atual: 3
Ponto! Pontuação atual: 4
Ponto! Pontuação atual: 5
Ponto! Pontuação atual: 6
Ponto! Pontuação atual: 7
Ponto! Pontuação atual: 8
Ponto! Pontuação atual: 9
Ponto! Pontuação atual: 10
Ponto! Pontuação atual: 11
Ponto! Pontuação atual: 12
Ponto! Pontuação atual: 13
Ponto! Pontuação atual: 14
Ponto! Pontuação atual: 15
Ponto! Pontuação atual: 16
